[XGBoost](https://www.kaggle.com/code/alexisbcook/xgboost#Example)  
구조화된 데이터를 위한 가장 정확한 모델링 기술

이 튜토리얼에서는 Gradient Boosting을 사용해 모델 구축 및 최적화 방법을 배운다. 이 방법은 kaggle 대회에서 많이 사용되고, 우수한 결과를 도출한다.

# Introduction
이 과정의 대부분은 **RandomForest(이하 'RF')** 방법으로 예측을 수행했는데, 이 방법은 많은 의사 결정 트리(Decision Tree, 이하'DT')의 예측을 평균화해 단일 DT보다 더 나은 성능을 달성한다. 

RF 방법을 'ensemble'이라 하며, <u>정의에 따르면 ensemble은 여러 모델(예:RF의 경우 여러 트리)의 예측을 결합한다.</u> 

다음으로 **Gradient Boosting(이하 'GB')**이라는 다른 앙상블 방법에 대해 알아보자. 

# Gradinet Boosting
**GB**은 사이클을 거쳐 모델을 앙상블에 추가하는 방법이다. 

이는 예측이 매우 naive(순진)할 수 있는 단일 모델로 앙상블을 초기화하는 것으로 시작된다.  
(설령 그것의 예측이 매우 부정확하더라도 앙상블에 대한 후속 추가가 이러한 오류를 해결할 것이다.)

이제, 사이클을 시작해보자.
* 먼저, 현재 앙상블을 사용해 데이터 세트의 각 관찰에 대한 예측을 생성한다.  
  예측을 위해 앙상블의 모든 모델의 예측을 추가한다.


* 이 예측들은 손실 함수(loss function)를 계산하는데 사용된다(손실함수 예-평균제곱오차)  


* 그 다음, 손실 함수를 사용해 앙상블에 추가될 새로운 모델을 맞춘다(fit).  
  특히, 이 새 모델을 앙상블에 추가하면 손실이 줄어들도록 모델 매개변수를 결정한다.  
  (참고:'GB'의 Gradient는 이 새 모델의 매개변수를 결정하기위해 손실 함수에 대한 경사(Gradient) 하강법을 사용할 것이라는 사실을 나타낸다.)
  
  
* 마지막으로, 새로운 모델을 앙상블에 추가한다. 


* 이후는 반복..

![nn](https://i.imgur.com/MvCGENh.png)

# Example

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data=pd.read_csv('./data/melb_data.csv')

# Select subset of predictors
cols_to_use=['Rooms','Distance','Landsize','BuildingArea','YearBuilt']
X=data[cols_to_use]

# Select target
y=data.Price

# Separate data into training and validation sets
X_train,X_valid,y_train,y_valid=train_test_split(X, y)

예시는 XGBoost 라이브러리로 작업한다.  


**XGBoost**는 **extreme gradient boosting**의 약자고, 성능 및 속도에 초점을 맞춘 몇 가지 추가 기능을 갖춘 GB을 구현한 것이다.  
(Scikit-learn은 GB의 다른 버전을 갖지만 XGBoost 는 몇 가지 기술적 이점을 가지고 있다.)  


다음 코드 셀에서 XGBoost**(xgboost.XGBRegressor)**를 위한 scikit-learn API를 가져온다.  
이는 우리가 scikit-learn에서처럼 모델을 만들고 맞출 수 있게 해준다.  
출력에서 알 수 있듯이 **XGBReggressor** 클래스에는 조정 가능한 매개 변수가 많으며 곧 이들에 대해 알게 될 것이다. 

In [7]:
from xgboost import XGBRegressor

model=XGBRegressor()
model.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

## 모델 예측 및 평가

In [8]:
from sklearn.metrics import mean_absolute_error as mae

pred=model.predict(X_valid)
print(f"Mean Absolute Error : {str(mae(pred, y_valid))}")

Mean Absolute Error : 238878.7804169735


# Parameter Tunning
xgboost는 정확도와 훈련 속도에 크게 영향을 미칠 수 있는 몇가지 매개 변수가 있다.  
먼저 알아야 할 파라미터는 다음과 같다.  

## n_estimaotrs
**n_estimators**는 위에서 설명한 모델링 주기를 몇 번 거치는지 지정한다. (앙상블에 포함된 모델의 수와 같다)
* 값이 너무 낮으면 underfitting이 발생해 훈련/테스트 데이터 모두에 대한 예측이 정확하지 않게 된다. 
* 값이 너무 높으면 과적합이 발생해 훈련 데이터에 대해 정확한 예측이 가능하나, 테스트 데이터에 대한 부정확한 예측이 발생한다. 

일반적인 값의 범위는 100-1000이지만 아래에 설명될 **learning_rate** 매개변수에 따라 크게 달라진다.

In [9]:
# 앙상블의 모델 수를 설정하는 코드는 다음과 같다
model=XGBRegressor(n_estimators=500)
model.fit(X_train, y_train)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

## early_stopping_rounds
**early_stopping_rounds**은 *n_estimators*에 대한 이상적인 값을 자동으로 찾는 방법을 제공한다.  
**early_stopping_rounds**는 *n_estimators*에 대한 강한 정지가 없더라고 유효성 검사 점수가 향상되지 않을 때 모형이 반복되지 않도록 한다.  
*n_estimators*에 높은 값을 설정한 다음, **early_stopping_rounds**를 사용해 반복을 중지할 최적의 타이밍을 찾는 것이 현명하다. 


랜덤 확률로 인해 검증 점수가 향상되지 않는 단일 라운드가 발생할 수 있므로 중지하기 전 허용할 직선 저하 라운드 수를 지정해야 한다.  
**early_stopping_rounds=5**를 설정하는 것은 합리적이며, 이 경우 5회 연속 악화되는 유효성 검사 점수 출력 후 중지한다.   

**early_stopping_rounds**를 사용할 때는 유효성 검사 점수를 계산하기 위해 일부 데이터를 별도로 설정해야 한다.   
이는 *eval_set* 매개 변수를 설정해 수행된다. 

In [11]:
# 조기 중기를 포함하도록 위의 예시 수정
model=XGBRegressor(n_estimators=500)
model.fit(X_train, y_train,
         early_stopping_rounds=5,
         eval_set=[(X_valid,y_valid)],
         verbose=False)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

## learning_rate
단순히 각 구성 요소 모델의 예측을 더해 예측을 얻는 대신 각 모델의 예측울 추가하기 전 작은 숫자(**learning rate=학습률**이라 한다)로 곱할 수 있다.  

이는 우리가 앙상블에 추가하는 각 나무(tree)가 우리에게 덜 도움이 된다는 것을 의미한다. 
따라서 과적합 없이 *n_estimators*에 더 높은 값을 설정할 수 있다.  
조기 중단(early stopping)을 사용하면 적절한 나무 수가 자동으로 결정된다.  

일반적으로 학습률이 낮고 추정기(estimator)가 많을수록 더 정확한 XGBoost 모델을 산출할 수 있지만, 모델을 훈련시키는데는 더 많은 시간이 소요될 것이며 이는 사이클을 통해 더 많은 반복을 하기 때문이다.  
기본적으로 XGBoost는 *learning_rate=0.1*로 설정한다. 

In [12]:
model=XGBRegressor(n_estimators=1000, learning_rate=0.05)
model.fit(X_train, y_train,
         early_stopping_rounds=5,
         eval_set=[(X_valid,y_valid)],
         verbose=False)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

## n_jobs
런타임이 고려되는 대규모 데이터 세트에서는 병렬 처리를 사용해 모델을 더 빠르게 빌드할 수 있다.  
**n_jobs** 매개변수를 시스템의 코어 수와 동일하게 설정하는 것이 일반적이다.  
더 작은 데이터 세트에서는 도움이 되지 않는다. 

결과 모델은 더 이상 개선되지 않으므로 피팅 시간에 대한 미세 최적화는 일반적으로 주의를 산만하게 할 뿐이다.  
그러나 적합(*fit*) 명령 동안 대기하는데 오랜 시간을 소비해야하는 대규모 데이터 세트에서 유용하다.

In [15]:
model=XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=8)
model.fit(X_train, y_train,
         early_stopping_rounds=5,
         eval_set=[(X_valid,y_valid)],
         verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

# Conclusion
[XGBoost](https://xgboost.readthedocs.io/en/latest/)는 표준 테이블 형식 데이터(이미지 및 비디오와 같은 좀 더 특이한 유형의 데이터와 대조적으로 DataFrames에 저장하는 데이터 유형)로 작업하기 위한 최고의 소프트웨어 라이브러리이다. 신중한 매개변수 조정을 통해 매우 정확한 모델을 훈련할 수 있다. 